In [8]:
import torch
import sys, os
sys.path.append("..")
from torch_geometric.datasets import DBP15K
from matching.models import HyperbolicRelCNN
from manifolds.hyperboloid import Hyperboloid
from KFoldAssessment import KFoldAssessment
from HoldOutSelection import HoldOutSelector
from dataset import GraphMatchingDataset

In [10]:
device='cuda'

In [13]:

class SumEmbedding(object):
    def __call__(self, data):
        data.x1, data.x2 = data.x1.sum(dim=1), data.x2.sum(dim=1)
        return data
path = os.path.join('..', '..', 'data', 'DBP15K')
data = DBP15K(path, 'fr_en', transform=SumEmbedding())[0].to(device)



gt = torch.cat([data.train_y, data.test_y], dim=-1)
data = GraphMatchingDataset('fr_en', data.x1/15, data.edge_index1, None, data.x2/15, data.edge_index2, None, gt)

In [14]:
parameter_ranges = {'space': ['Hyperbolic'],
                    'c': [1],
                    'in_channels': [50],
                    'out_channels': [50],
                    'num_layers': [1],
                    'cat': [True, False],
                    'lin': [True],
                    'dropout': [0],
                    'sim': ['dot'],
                    'k': [10],
                    'lr': [3e-4, 1e-3],
                    'gamma': [0.7],
                    'max_epochs': [200]
                   }

ass = KFoldAssessment(5, 'hyp', parameter_ranges, num_configs=2)

ass.risk_assessment(data, device='cuda')

START OF FOLD 1
tensor(1.9747, device='cuda:0') 0
@1: 0.6871502249533633 0 config done tensor(1.8448, device='cuda:0') 0
@1: 0.48140019752002633 1 config done PROCESSING RESULTS
Model selection winner for experiment hyp/5_CV/FOLD_1: 1 with hits1 0.687


KeyboardInterrupt: 

In [9]:
class Config:
    def __init__(self, space, c, in_channels, out_channels,
                num_layers, cat, lin, dropout, sim, k, lr, gamma, max_epochs):
        self.space = space
        self.c = c
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_layers = num_layers
        self.cat = cat
        self.lin = lin
        self.dropout = dropout
        self.sim = sim
        self.k = k
        self.lr = lr
        self.gamma = gamma
        self.max_epochs = max_epochs
        
    def __repr__(self):
        return f'{self.space} {self.c}; in_channels={self.in_channels}, \
        out_channels={self.out_channels}, num_layers={self.num_layers}, sim={self.sim}; cat={self.cat}, \
        lin={self.lin}; k={self.k}, lr={self.lr}, gamma={self.gamma}, max_epochs={self.max_epochs}'
        
        
c = Config(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 0)